In [95]:
import re
import pandas as pd
import weapon_dict
import secret
pd.set_option('display.max_rows', None)

In [96]:
file = open(f"{secret.location}\Cataphract CTF-2X.txt").read()
list_str = file.split('\n')
list_str = [x for x in list_str if x != '']

In [97]:
attributes = []
weapons = []
values_interest = ['Cost','Technology Base','Tonnage:','Battle Value']
for i in range(len(list_str)):
    for x in values_interest:
        if x in list_str[i]:
            attributes.append(list_str[i][(len(x+' ')):])
            
attributes = [s.strip() for s in attributes]
attribute_dictionary = dict(zip(values_interest, attributes))           

In [98]:
attribute_dictionary

{'Cost': '5,877,353 C-bills',
 'Technology Base': 'Inner Sphere (Introductory)',
 'Tonnage:': '70',
 'Battle Value': '1,344'}

In [99]:
# Movement dict creation and calculation
movement_substr = [i for i in list_str if 'MP' in i]
movement_value = [i * 2 for i in ([int(movement_substr[0][13:14]),int(movement_substr[2][13:14])])]
movement_key = ['walking', 'jumping']
movement_dictionary = dict(zip(movement_key, movement_value))

In [100]:
movement_dictionary

{'walking': 8, 'jumping': 0}

In [101]:
# Grabbing the cost elements
key_value_pairs = {}
current_key = None

lines = file.split('\n')
for line in lines:
    line = line.strip()
    if line:
        if ':' in line:
            current_key, value = line.split(':', 1)
            key_value_pairs[current_key.strip()] = value.strip()
        elif current_key is not None:
            if current_key == 'Armaments':
                if current_key not in key_value_pairs:
                    key_value_pairs[current_key] = line.strip()
                else:
                    key_value_pairs[current_key] += f', {line.strip()}'
            else:
                key_value_pairs[current_key] += f', {line.strip()}'

In [102]:
# Creating the weapon dictionary
keys = ['Armament']
dict2 = {x:key_value_pairs[x] for x in keys}
weapons = dict2['Armament'].split(',')

def strip_list_noempty(mylist):
    newlist = (item.strip() if hasattr(item, 'strip') else item for item in mylist)
    return [item for item in newlist if item != '']

weapons = strip_list_noempty(weapons)

weapon_value = []
weapon_key = []
for i in weapons:
    weapon_key.append(i[2:])
    weapon_value.append(i[:1])
weapon_dict = dict(zip(weapon_key, weapon_value))

In [103]:
weapon_dict

{'Medium Laser': '2', 'SRM 4': '1', 'Large Laser': '1', 'AC/10': '1'}

In [104]:
list_str

['Cataphract CTF-2X',
 'Mass: 70 tons',
 'Chassis: Standard Biped',
 'Power Plant: 280 Fusion',
 'Cruising Speed: 43.2 kph',
 'Maximum Speed: 64.8 kph',
 'Jump Jets: None',
 '     Jump Capacity: 0 meters',
 'Armor: Standard',
 'Armament:',
 '     2 Medium Laser',
 '     1 SRM 4',
 '     1 Large Laser',
 '     1 AC/10',
 'Manufacturer: Unknown',
 '     Primary Factory: Unknown',
 'Communication System: Unknown',
 'Targeting & Tracking System: Unknown',
 'Introduction Year: 3027',
 'Tech Rating/Availability: D/X-E-D-D',
 'Cost: 5,877,353 C-bills',
 'Type: Cataphract',
 'Technology Base: Inner Sphere (Introductory) ',
 'Tonnage: 70',
 'Battle Value: 1,344',
 'Equipment                                          Mass',
 'Internal Structure                                    7',
 'Engine                        280 Fusion             16',
 '\tWalking MP: 4',
 '\tRunning MP: 6',
 '\tJumping MP: 0',
 'Heat Sink                     14                      4',
 'Gyro                               